# ENGR-E 533: Deep Learning Systems
## Homework 2

### Khandokar Md. Nayem (knayem@iu.edu)
### Apr 3, 2018

### Import necessary files and set environment parameters
My assigned Node is `r-005` and GPU `0`.

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional
from keras.models import Sequential
from keras.layers import Dense, Activation, TimeDistributed
from keras.utils.np_utils import to_categorical

import tensorflow as tf
import numpy as np
import librosa
import glob

Using TensorFlow backend.


### Global Variables

In [2]:
PATH_directory = '/N/u/knayem/data/timit-homework/'

PATH_train = 'tr/'
PATH_val = 'v/'
PATH_test = 'te/'

CLEAN_format_train = 'trs*.wav'
NOISE_format_train = 'trn*.wav'
MIX_format_train = 'trx*.wav'

CLEAN_format_val = 'vs*.wav'
NOISE_format_val = 'vn*.wav'
MIX_format_val = 'vx*.wav'

MIX_format_test = 'tex*.wav'

Max_RNN = 5

### File Names

In [3]:
TRAIN_S = 'TRAIN_S.txt'
TRAIN_N = 'TRAIN_N.txt'
TRAIN_X_cmplx = 'TRAIN_X_cmplx.txt'
TRAIN_X = 'TRAIN_X.txt'

VAL_S = 'VAL_S.txt'
VAL_N = 'VAL_N.txt'
VAL_X_cmplx = 'VAL_X_cmplx.txt'
VAL_X = 'VAL_X.txt'


TEST_S = 'TEST_S.txt'
TEST_N = 'TEST_N.txt'
TEST_X_cmplx = 'TEST_X_cmplx.txt'
TEST_X = 'TEST_X.txt'


A function to get the Magnitude Spectum of a file. 

In [10]:
def preprossed_data(file_name):
    
    sn, sr=librosa.load(file_name, sr=None)
    X=librosa.stft(sn, n_fft=1024, hop_length=512)
    X_mag = np.abs(X)
#     print('X_mag',X_mag.shape)
    return X, X_mag

### Save Train Data  

### Save Validate Data

In [8]:
with open(VAL_S,'wb') as fs, open(VAL_N,'wb') as fn, open(VAL_X_cmplx,'wb') as fx_cmplx, open(VAL_X,'wb') as fx: 
    
    for file_s, file_n, file_x in zip(sorted(glob.glob(PATH_directory+PATH_val+CLEAN_format_val)),sorted(glob.glob(PATH_directory+PATH_val+NOISE_format_val)),sorted(glob.glob(PATH_directory+PATH_val+MIX_format_val))):
        _,s = preprossed_data(file_s)
        np.savetxt(fs, s, fmt='%.5f')
        fs.write(b'\n')
        
        _,n = preprossed_data(file_n)
        np.savetxt(fn, n, fmt='%.5f')
        fn.write(b'\n')
        
        x_cmplx,x = preprossed_data(file_x)
        np.savetxt(fx, x, fmt='%.5f')
        fx.write(b'\n')
        
        np.savetxt(fx_cmplx, x_cmplx, fmt='%.5f')
        fx_cmplx.write(b'\n')
        
fs.close()
fn.close()
fx_cmplx.close()
fx.close()

### Save Test Data

In [12]:
with open(TEST_S,'wb') as fs, open(TEST_N,'wb') as fn, open(TEST_X_cmplx,'wb') as fx_cmplx, open(TEST_X,'wb') as fx: 
    count = 1
    for file_x in sorted(glob.glob(PATH_directory+PATH_test+MIX_format_test)):
        x_cmplx,x = preprossed_data(file_x)
        np.savetxt(fx, x, fmt='%.5f')
        fx.write(b'\n')
        
        np.savetxt(fx_cmplx, x_cmplx, fmt='%.5e%+.5ej')
        fx_cmplx.write(b'\n')
        
        print(count, '...')
        count +=1
        
fx_cmplx.close()
fx.close()

ValueError: fmt has wrong number of % formats:  %.5e%+.5ej

### Load Data

In [4]:
def load_file(file_name):
    
    with open(file_name) as f:
        lines=f.readlines()
        print(len(lines))
        sentence_full=[]
        count = 0
        sentence=[]
        for line in lines:

            if count < 513:
                if count ==0:
                    sentence=np.array(np.fromstring(line, dtype=float, sep=' '), ndmin=2)
                    count+=1
                else:
                    myarray = np.array(np.fromstring(line, dtype=float, sep=' '), ndmin=2)
                    sentence=np.concatenate((sentence, myarray), axis=0)
                    count+=1
            else:
                sentence_full.append(sentence) 
                count=0
                sentence=[]
                
        return sentence_full

In [5]:
DATA_train_s = load_file(TRAIN_S)
DATA_train_n = load_file(TRAIN_N)
DATA_train_x_cmplx = load_file(TRAIN_X_cmplx)
DATA_train_x = load_file(TRAIN_X)

616800
616800
616800
616800


In [7]:
DATA_train_s[0].shape

(513, 65)

In [6]:
DATA_val_s = load_file(VAL_S)
DATA_val_n = load_file(VAL_N)
DATA_val_x_cmplx = load_file(VAL_X_cmplx)
DATA_val_x = load_file(VAL_X)

616800
616800
616800
616800


In [7]:
DATA_test_x_cmplx = load_file(TEST_X_cmplx)
DATA_test_x = load_file(TEST_X)

205600
205600


In [8]:
DATA_train_M=[ (1.0*(DATA_train_s[i]>DATA_train_n[i])) for i in range(len(DATA_train_s)) ]

DATA_val_M=[ (1.0*(DATA_val_s[i]>DATA_val_n[i])) for i in range(len(DATA_val_s)) ]
    

### Batch Generator

In [9]:
def next_batch_mb(X_,Y_,mini_batch):
    
    batch_x, batch_y = None, None
    
    for e,(x,y) in enumerate(zip(X_,Y_)):
#         print(e)
        
        batch_x = np.array(x.T) if batch_x is None else np.concatenate( (batch_x,x.T), axis=0)
        batch_y = np.array(y.T) if batch_y is None else np.concatenate( (batch_y,y.T), axis=0)
        
#         print('e',e,'batch_x',batch_x.shape,'batch_y',batch_y.shape)
#         print('batch_y',batch_y.shape)
        
        if e>0 and (e+1)%10==0:
#             print('ZZZ--> e',e,'batch_x',batch_x.shape,'batch_y',batch_y.shape)
            
            batch_x, temp_x = None, batch_x
            batch_y, temp_y = None, batch_y
            
            temp_x = temp_x.reshape((-1,Max_RNN,513))
            temp_y = temp_y.reshape((-1,Max_RNN,513))
            
            total_mini_batch = temp_x.shape[0]//mini_batch
#             print('temp_x.shape[0]',temp_x.shape[0], 'total_mini_batch',total_mini_batch)
            
            for mb in range(total_mini_batch):
                start_b = (mb*mini_batch)
                end_b = ((mb+1)*mini_batch)

                yield temp_x[start_b:end_b],temp_y[start_b:end_b]

    

In [10]:
def next_batch(X_,Y_):
    
    batch_x, batch_y = None, None
    
    for e,(x,y) in enumerate(zip(X_,Y_)):
#         print(e)
        
        batch_x = np.array(x.T) if batch_x is None else np.concatenate( (batch_x,x.T), axis=0)
        batch_y = np.array(y.T) if batch_y is None else np.concatenate( (batch_y,y.T), axis=0)
        
#         print('batch_x',batch_x.shape)
#         print('batch_y',batch_y.shape)
        
        if e>0 and (e+1)%10==0:
            temp_x, batch_x = batch_x, None
            temp_y, batch_y = batch_y, None
            
            temp_x = temp_x.reshape((-1,Max_RNN,513))
            temp_y = temp_y.reshape((-1,Max_RNN,513))

#             print('temp_x',temp_x.shape)
#             print('temp_y',temp_y.shape)
        
            yield temp_x,temp_y


In [11]:
c=0
for p,q in next_batch(DATA_train_x, DATA_train_M):
    print('p',p.shape)
#     print(p)
    print('q',q.shape)
#     print(q)
    c+=1
    
    if c == 2:
        break
print(c)

batch_x (65, 513)
batch_y (65, 513)
batch_x (130, 513)
batch_y (130, 513)
batch_x (195, 513)
batch_y (195, 513)
batch_x (260, 513)
batch_y (260, 513)
batch_x (325, 513)
batch_y (325, 513)
batch_x (390, 513)
batch_y (390, 513)
batch_x (455, 513)
batch_y (455, 513)
batch_x (520, 513)
batch_y (520, 513)
batch_x (585, 513)
batch_y (585, 513)
batch_x (650, 513)
batch_y (650, 513)
p (130, 5, 513)
q (130, 5, 513)
batch_x (130, 513)
batch_y (130, 513)
batch_x (260, 513)
batch_y (260, 513)
batch_x (390, 513)
batch_y (390, 513)
batch_x (520, 513)
batch_y (520, 513)
batch_x (650, 513)
batch_y (650, 513)
batch_x (780, 513)
batch_y (780, 513)
batch_x (910, 513)
batch_y (910, 513)
batch_x (1040, 513)
batch_y (1040, 513)
batch_x (1170, 513)
batch_y (1170, 513)
batch_x (1300, 513)
batch_y (1300, 513)
p (260, 5, 513)
q (260, 5, 513)
2


In [20]:
len(DATA_train_x)
DATA_train_x[12].shape

(513, 130)

In [23]:
model = Sequential()

model.add(Bidirectional(GRU(Max_RNN, return_sequences=True), input_shape=(Max_RNN,513)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(Max_RNN, return_sequences=True)))
# model.add(GRU(output_dim = 513, input_length = 5, input_dim = 513, return_sequences=True))

# model.add(Activation('relu'))
# model.add(TimeDistributed(Dense(513, activation='sigmoid')))
model.add(Dense(513, activation='sigmoid'))

model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

for e in range(100):
    for (b_x,b_y), (v_x,v_y) in zip(next_batch(DATA_train_x, DATA_train_M), next_batch(DATA_val_x, DATA_val_x)):
        model.fit(b_x, b_y, validation_data=(v_x,v_y), shuffle=True, batch_size=100)

#     model.fit( , epochs=20, steps_per_epoch=700, validation_data=next_batch_mb(DATA_val_x, DATA_val_x,10), validation_steps=700)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_6 (Bidirection (None, 5, 10)             15570     
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 10)             0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 5, 10)             480       
_________________________________________________________________
dense_3 (Dense)              (None, 5, 513)            5643      
Total params: 21,693
Trainable params: 21,693
Non-trainable params: 0
_________________________________________________________________
None
Train on 130 samples, validate on 130 samples
Epoch 1/1
130/130 [==============================] - 2s 19ms/step - loss: 0.2501 - acc: 0.0000e+00 - val_loss: 0.2231 - val_acc: 0.0015
Train on 260 samples, validate on 260 samples
Epoch 1/1
260/260 [==================

212/212 [==============================] - 0s 697us/step - loss: 0.2415 - acc: 0.0028 - val_loss: 0.1712 - val_acc: 0.0000e+00
Train on 210 samples, validate on 210 samples
Epoch 1/1
210/210 [==============================] - 0s 666us/step - loss: 0.2441 - acc: 9.5238e-04 - val_loss: 0.1768 - val_acc: 0.0000e+00
Train on 158 samples, validate on 158 samples
Epoch 1/1
158/158 [==============================] - 0s 607us/step - loss: 0.2321 - acc: 0.0013 - val_loss: 0.1764 - val_acc: 0.0000e+00
Train on 198 samples, validate on 198 samples
Epoch 1/1
198/198 [==============================] - 0s 542us/step - loss: 0.2323 - acc: 0.0040 - val_loss: 0.1745 - val_acc: 0.0010
Train on 206 samples, validate on 206 samples
Epoch 1/1
206/206 [==============================] - 0s 697us/step - loss: 0.2457 - acc: 0.0019 - val_loss: 0.1791 - val_acc: 0.0000e+00
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 681us/step - loss: 0.2359 - acc: 0.0033

174/174 [==============================] - 0s 517us/step - loss: 0.2367 - acc: 0.0195 - val_loss: 0.2232 - val_acc: 0.0000e+00
Train on 140 samples, validate on 140 samples
Epoch 1/1
140/140 [==============================] - 0s 645us/step - loss: 0.2373 - acc: 0.0229 - val_loss: 0.2139 - val_acc: 0.0043
Train on 228 samples, validate on 228 samples
Epoch 1/1
228/228 [==============================] - 0s 590us/step - loss: 0.2458 - acc: 0.0272 - val_loss: 0.2193 - val_acc: 0.0000e+00
Train on 202 samples, validate on 202 samples
Epoch 1/1
202/202 [==============================] - 0s 655us/step - loss: 0.2246 - acc: 0.0089 - val_loss: 0.1943 - val_acc: 9.9010e-04
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 526us/step - loss: 0.2289 - acc: 0.0044 - val_loss: 0.2272 - val_acc: 0.0000e+00
Train on 242 samples, validate on 242 samples
Epoch 1/1
242/242 [==============================] - 0s 733us/step - loss: 0.2462 - acc: 0.0157 - v

202/202 [==============================] - 0s 1ms/step - loss: 0.2006 - acc: 0.0000e+00 - val_loss: 0.2180 - val_acc: 9.9010e-04
Train on 148 samples, validate on 148 samples
Epoch 1/1
148/148 [==============================] - 0s 967us/step - loss: 0.2087 - acc: 0.0014 - val_loss: 0.2472 - val_acc: 0.0014
Train on 252 samples, validate on 252 samples
Epoch 1/1
252/252 [==============================] - 0s 893us/step - loss: 0.2182 - acc: 7.9365e-04 - val_loss: 0.2204 - val_acc: 0.0016
Train on 152 samples, validate on 152 samples
Epoch 1/1
152/152 [==============================] - 0s 576us/step - loss: 0.2106 - acc: 0.0000e+00 - val_loss: 0.2186 - val_acc: 0.0039
Train on 118 samples, validate on 118 samples
Epoch 1/1
118/118 [==============================] - 0s 775us/step - loss: 0.2079 - acc: 0.0017 - val_loss: 0.1881 - val_acc: 0.0034
Train on 232 samples, validate on 232 samples
Epoch 1/1
232/232 [==============================] - 0s 653us/step - loss: 0.2073 - acc: 0.0017 - val

Train on 206 samples, validate on 206 samples
Epoch 1/1
206/206 [==============================] - 0s 1ms/step - loss: 0.1915 - acc: 9.7087e-04 - val_loss: 0.1866 - val_acc: 0.0068
Train on 288 samples, validate on 288 samples
Epoch 1/1
288/288 [==============================] - 0s 738us/step - loss: 0.2110 - acc: 0.0014 - val_loss: 0.1959 - val_acc: 0.0063
Train on 222 samples, validate on 222 samples
Epoch 1/1
222/222 [==============================] - 0s 732us/step - loss: 0.1912 - acc: 0.0000e+00 - val_loss: 0.1778 - val_acc: 0.0171
Train on 288 samples, validate on 288 samples
Epoch 1/1
288/288 [==============================] - 0s 520us/step - loss: 0.1850 - acc: 0.0014 - val_loss: 0.1669 - val_acc: 0.0069
Train on 78 samples, validate on 78 samples
Epoch 1/1
78/78 [==============================] - 0s 597us/step - loss: 0.2022 - acc: 0.0026 - val_loss: 0.1904 - val_acc: 0.0051
Train on 164 samples, validate on 164 samples
Epoch 1/1
164/164 [==============================] - 0s 5

...

264/264 [==============================] - 0s 525us/step - loss: 0.1717 - acc: 0.0167 - val_loss: 0.2057 - val_acc: 0.0068
Train on 200 samples, validate on 200 samples
Epoch 1/1
200/200 [==============================] - 0s 480us/step - loss: 0.1534 - acc: 0.0510 - val_loss: 0.2352 - val_acc: 0.0100
Train on 206 samples, validate on 206 samples
Epoch 1/1
206/206 [==============================] - 0s 635us/step - loss: 0.1431 - acc: 0.0398 - val_loss: 0.2597 - val_acc: 0.0029
Train on 288 samples, validate on 288 samples
Epoch 1/1
288/288 [==============================] - 0s 471us/step - loss: 0.1795 - acc: 0.0375 - val_loss: 0.2276 - val_acc: 0.0076
Train on 222 samples, validate on 222 samples
Epoch 1/1
222/222 [==============================] - 0s 595us/step - loss: 0.1572 - acc: 0.0423 - val_loss: 0.2375 - val_acc: 0.0153
Train on 288 samples, validate on 288 samples
Epoch 1/1
288/288 [==============================] - 0s 490us/step - loss: 0.1536 - acc: 0.0278 - val_loss: 0.2020 

186/186 [==============================] - 0s 492us/step - loss: 0.1611 - acc: 0.0290 - val_loss: 0.2309 - val_acc: 0.0054
Train on 216 samples, validate on 216 samples
Epoch 1/1
216/216 [==============================] - 0s 608us/step - loss: 0.1526 - acc: 0.0287 - val_loss: 0.2942 - val_acc: 0.0139
Train on 150 samples, validate on 150 samples
Epoch 1/1
150/150 [==============================] - 0s 608us/step - loss: 0.1520 - acc: 0.0387 - val_loss: 0.2704 - val_acc: 0.0107
Train on 132 samples, validate on 132 samples
Epoch 1/1
132/132 [==============================] - 0s 655us/step - loss: 0.1465 - acc: 0.0288 - val_loss: 0.1989 - val_acc: 0.0030
Train on 280 samples, validate on 280 samples
Epoch 1/1
280/280 [==============================] - 0s 545us/step - loss: 0.1451 - acc: 0.0279 - val_loss: 0.2497 - val_acc: 0.0021
Train on 148 samples, validate on 148 samples
Epoch 1/1
148/148 [==============================] - 0s 604us/step - loss: 0.1824 - acc: 0.0203 - val_loss: 0.2215 

196/196 [==============================] - 0s 459us/step - loss: 0.1510 - acc: 0.0265 - val_loss: 0.2439 - val_acc: 0.0020
Train on 148 samples, validate on 148 samples
Epoch 1/1
148/148 [==============================] - 0s 574us/step - loss: 0.1511 - acc: 0.0311 - val_loss: 0.2340 - val_acc: 0.0027
Train on 234 samples, validate on 234 samples
Epoch 1/1
234/234 [==============================] - 0s 546us/step - loss: 0.1641 - acc: 0.0376 - val_loss: 0.2422 - val_acc: 0.0043
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 488us/step - loss: 0.1727 - acc: 0.0300 - val_loss: 0.3146 - val_acc: 0.0078
Train on 130 samples, validate on 130 samples
Epoch 1/1
130/130 [==============================] - 0s 610us/step - loss: 0.1380 - acc: 0.0323 - val_loss: 0.2401 - val_acc: 0.0262
Train on 260 samples, validate on 260 samples
Epoch 1/1
260/260 [==============================] - 0s 547us/step - loss: 0.1459 - acc: 0.0431 - val_loss: 0.2505 

158/158 [==============================] - 0s 549us/step - loss: 0.1582 - acc: 0.0380 - val_loss: 0.2550 - val_acc: 0.0025
Train on 198 samples, validate on 198 samples
Epoch 1/1
198/198 [==============================] - 0s 467us/step - loss: 0.1716 - acc: 0.0394 - val_loss: 0.2613 - val_acc: 0.0040
Train on 206 samples, validate on 206 samples
Epoch 1/1
206/206 [==============================] - 0s 638us/step - loss: 0.1685 - acc: 0.0369 - val_loss: 0.2884 - val_acc: 0.0039
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 491us/step - loss: 0.1559 - acc: 0.0500 - val_loss: 0.2543 - val_acc: 0.0033
Train on 214 samples, validate on 214 samples
Epoch 1/1
214/214 [==============================] - 0s 629us/step - loss: 0.1563 - acc: 0.0308 - val_loss: 0.2543 - val_acc: 0.0159
Train on 186 samples, validate on 186 samples
Epoch 1/1
186/186 [==============================] - 0s 475us/step - loss: 0.1643 - acc: 0.0484 - val_loss: 0.2449 

202/202 [==============================] - 0s 606us/step - loss: 0.1461 - acc: 0.0307 - val_loss: 0.2443 - val_acc: 0.0158
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 572us/step - loss: 0.1511 - acc: 0.0178 - val_loss: 0.2726 - val_acc: 0.0033
Train on 242 samples, validate on 242 samples
Epoch 1/1
242/242 [==============================] - 0s 552us/step - loss: 0.1575 - acc: 0.0264 - val_loss: 0.3363 - val_acc: 0.0124
Train on 144 samples, validate on 144 samples
Epoch 1/1
144/144 [==============================] - 0s 568us/step - loss: 0.1707 - acc: 0.0236 - val_loss: 0.2680 - val_acc: 0.0014
Train on 356 samples, validate on 356 samples
Epoch 1/1
356/356 [==============================] - 0s 501us/step - loss: 0.1520 - acc: 0.0213 - val_loss: 0.2835 - val_acc: 0.0067
Train on 234 samples, validate on 234 samples
Epoch 1/1
234/234 [==============================] - 0s 557us/step - loss: 0.1470 - acc: 0.0282 - val_loss: 0.3670 

In [24]:
# Final evaluation of the model
scores = []
for v_x,v_y in next_batch(DATA_val_x, DATA_val_x):
    scores.append( model.evaluate(v_x, v_y,verbose=0)[1] )

print("Accuracy: %.2f%%" % np.mean(scores*100))

Accuracy: 0.01%


In [ ]:
def next_batchX(X_):
    
    batch_x = None
    
    for e,x in enumerate(X_): 
        batch_x = np.array(x.T) if batch_x is None else np.concatenate( (batch_x,x.T), axis=0)
        
        if e>0 and (e+1)%10==0:
            temp_x, batch_x = batch_x, None
            
            temp_x = temp_x.reshape((-1,Max_RNN,513))

            yield temp_x

In [27]:
for te_x in DATA_test_x:
    labels = model.predict(te_x)
    labels.shape

ValueError: Error when checking : expected bidirectional_6_input to have 3 dimensions, but got array with shape (513, 110)

0
batch_x (65, 513)
batch_y (65, 513)
1
batch_x (65, 1026)
batch_y (65, 1026)
2
batch_x (65, 1539)
batch_y (65, 1539)
3
batch_x (65, 2052)
batch_y (65, 2052)
4
batch_x (65, 2565)
batch_y (65, 2565)
5
batch_x (65, 3078)
batch_y (65, 3078)
6
batch_x (65, 3591)
batch_y (65, 3591)
7
batch_x (65, 4104)
batch_y (65, 4104)
8
batch_x (65, 4617)
batch_y (65, 4617)
9
batch_x (65, 5130)
batch_y (65, 5130)
temp_x (130, 5, 513)
temp_y (130, 5, 513)
p (130, 5, 513)
[[[3.85350e-01 9.41800e-02 5.24800e-02 ... 1.36770e-01 8.30000e-04
   7.60900e-02]
  [4.26000e-02 1.56800e-02 1.00000e-03 ... 1.61000e-03 1.57000e-03
   1.98000e-03]
  [6.79800e-02 1.91600e-02 6.88900e-02 ... 1.50000e-03 1.54000e-03
   1.51000e-03]
  [8.71700e-02 2.69800e-02 6.13600e-02 ... 2.55000e-03 1.85000e-03
   1.25000e-03]
  [7.08370e-01 6.99290e-01 1.42582e+00 ... 2.20100e-02 1.78100e-02
   1.47000e-02]]

 [[2.28000e-03 2.22000e-03 2.25000e-03 ... 2.00000e-05 2.30000e-04
   5.90000e-04]
  [4.66200e-02 4.33500e-02 3.91600e-02 ... 1

Create IRM for training.

print('DATA_train_s',DATA_train_s.shape)
print('DATA_train_n',DATA_train_n.shape)
print('DATA_train_x',DATA_train_x.shape)

DATA_train_s = DATA_train_s.reshape( (513,5,-1))
DATA_train_n = DATA_train_n.reshape( (513,5,-1))
DATA_train_x = DATA_train_x.reshape( (513,5,-1))
DATA_train_M = 1*(DATA_train_s>DATA_train_n)

print('DATA_train_s',DATA_train_s.shape)
print('DATA_train_n',DATA_train_n.shape)
print('DATA_train_x',DATA_train_x.shape)
print('DATA_train_M',DATA_train_M.shape)

### Validate Data

DATA_val_s = None
DATA_val_n = None
DATA_val_x = None

for file_s, file_n, file_x in zip(sorted(glob.glob(PATH_directory+PATH_val+CLEAN_format_val)),sorted(glob.glob(PATH_directory+PATH_val+NOISE_format_val)),sorted(glob.glob(PATH_directory+PATH_val+MIX_format_val))):
    
    DATA_val_s = np.array(preprossed_data(file_s)) if DATA_val_s is None else np.concatenate( (DATA_val_s,preprossed_data(file_s)),axis=1)
    DATA_val_n = np.array(preprossed_data(file_n)) if DATA_val_n is None else np.concatenate( (DATA_val_n,preprossed_data(file_n)),axis=1)
    DATA_val_x = np.array(preprossed_data(file_x)) if DATA_val_x is None else np.concatenate( (DATA_val_x,preprossed_data(file_x)),axis=1)

print('DATA_val_s',DATA_val_s.shape)
print('DATA_val_n',DATA_val_n.shape)
print('DATA_val_x',DATA_val_x.shape)

DATA_val_s = DATA_val_s.reshape( (513,5,-1))
DATA_val_n = DATA_val_n.reshape( (513,5,-1))
DATA_val_x = DATA_val_x.reshape( (513,5,-1))
DATA_val_M = 1*(DATA_val_s>DATA_val_n)

print('DATA_val_s',DATA_val_s.shape)
print('DATA_val_n',DATA_val_n.shape)
print('DATA_val_x',DATA_val_x.shape)
print('DATA_val_M',DATA_val_M.shape)

### Test Data

DATA_test_x = None

for file_x in sorted(glob.glob(PATH_directory+PATH_test+MIX_format_test)):
    DATA_test_x = np.array(preprossed_data(file_x)) if DATA_test_x is None else np.concatenate( (DATA_test_x,preprossed_data(file_x)),axis=1)


In [ ]:
print('DATA_test_x',DATA_test_x.shape)
DATA_test_x = DATA_test_x.reshape( (513,5,-1))

print('DATA_test_x',DATA_test_x.shape)

In [7]:
def preprossed_dataX(file_name):
    
    sn, sr=librosa.load(file_name, sr=None)
    X=librosa.stft(sn, n_fft=1024, hop_length=512)
    return X

### Validate Data Complex

In [8]:
DATA_val_sX = None
DATA_val_xX = None

for file_s, file_x in zip( sorted(glob.glob(PATH_directory+PATH_val+CLEAN_format_val)), sorted(glob.glob(PATH_directory+PATH_val+MIX_format_val))):
    
    DATA_val_sX = np.array(preprossed_dataX(file_s)) if DATA_val_sX is None else np.concatenate( (DATA_val_sX,preprossed_dataX(file_s)),axis=1)
    DATA_val_xX = np.array(preprossed_dataX(file_x)) if DATA_val_xX is None else np.concatenate( (DATA_val_xX,preprossed_dataX(file_x)),axis=1)
    
    
    
   

In [9]:
print('DATA_val_sX',DATA_val_sX.shape)
print('DATA_val_xX',DATA_val_xX.shape)

DATA_val_sX = DATA_val_sX.reshape( (513,5,-1))
DATA_val_xX = DATA_val_xX.reshape( (513,5,-1))

print('DATA_val_sX',DATA_val_sX.shape)
print('DATA_val_xX',DATA_val_xX.shape)

DATA_val_sX (513, 118550)
DATA_val_xX (513, 118550)
DATA_val_sX (513, 5, 23710)
DATA_val_xX (513, 5, 23710)


In [10]:
np.save('DATA_val_sX.npy', DATA_val_sX)
np.save('DATA_val_xX.npy', DATA_val_xX)

In [50]:
DATA_val_sX_f = np.load('DATA_val_sX.npy')
DATA_val_xX_f = np.load('DATA_val_xX.npy')

DATA_test_x (513, 44150)
DATA_test_x (513, 5, 8830)


### Save Dataset in files

In [3]:
np.save('DATA_train_s.npy', DATA_train_s)
np.save('DATA_train_n.npy', DATA_train_n)
np.save('DATA_train_x.npy', DATA_train_x)
np.save('DATA_train_M.npy', DATA_train_M)


np.save('DATA_val_s.npy', DATA_val_s)
np.save('DATA_val_n.npy', DATA_val_n)
np.save('DATA_val_x.npy', DATA_val_x)
np.save('DATA_val_M.npy', DATA_val_M)

np.save('DATA_test_x.npy', DATA_test_x)

np.save('DATA_val_sX.npy', DATA_val_sX)
np.save('DATA_val_xX.npy', DATA_val_xX)

NameError: name 'DATA_train_s' is not defined

### Load Dataset in files

In [3]:
DATA_train_s_f = np.load('DATA_train_s.npy')
DATA_train_n_f = np.load('DATA_train_n.npy')
DATA_train_x_f = np.load('DATA_train_x.npy')
DATA_train_M_f = np.load('DATA_train_M.npy')

DATA_val_s_f = np.load('DATA_val_s.npy')
DATA_val_n_f = np.load('DATA_val_n.npy')
DATA_val_x_f = np.load('DATA_val_x.npy')
DATA_val_M_f = np.load('DATA_val_M.npy')

DATA_test_x_f = np.load('DATA_test_x.npy')

DATA_val_sX_f = np.load('DATA_val_sX.npy')
DATA_val_xX_f = np.load('DATA_val_xX.npy')

In [4]:
print('DATA_train_s_f', DATA_train_s_f.shape)
print('DATA_train_M_f', DATA_train_M_f.shape)
print('DATA_val_x_f', DATA_val_x_f.shape)
print('DATA_test_x_f', DATA_test_x_f.shape)
print('DATA_val_xX_f', DATA_val_xX_f.shape)

DATA_train_s_f (513, 5, 23710)
DATA_train_M_f (513, 5, 23710)
DATA_val_x_f (513, 5, 23710)
DATA_test_x_f (513, 5, 8830)
DATA_val_xX_f (513, 5, 23710)


In [5]:
DATA_train_s_f = DATA_train_s_f.reshape(513,-1)
print('DATA_train_s_f', DATA_train_s_f.shape)

DATA_train_s_f = DATA_train_s_f.T
print('DATA_train_s_f', DATA_train_s_f.shape)

DATA_train_s_f = DATA_train_s_f.reshape(-1,5,513)
print('DATA_train_s_f', DATA_train_s_f.shape)

DATA_train_s_f (513, 118550)
DATA_train_s_f (118550, 513)
DATA_train_s_f (23710, 5, 513)


In [6]:
DATA_train_n_f = (DATA_train_n_f.reshape(513,-1).T).reshape(-1,5,513)
DATA_train_x_f = (DATA_train_x_f.reshape(513,-1).T).reshape(-1,5,513)
DATA_train_M_f = (DATA_train_M_f.reshape(513,-1).T).reshape(-1,5,513)

DATA_val_s_f = (DATA_val_s_f.reshape(513,-1).T).reshape(-1,5,513)
DATA_val_n_f = (DATA_val_n_f.reshape(513,-1).T).reshape(-1,5,513)
DATA_val_x_f = (DATA_val_x_f.reshape(513,-1).T).reshape(-1,5,513)
DATA_val_M_f = (DATA_val_M_f.reshape(513,-1).T).reshape(-1,5,513)

DATA_test_x_f = (DATA_test_x_f.reshape(513,-1).T).reshape(-1,5,513)

DATA_val_sX_f = (DATA_val_sX_f.reshape(513,-1).T).reshape(-1,5,513)
DATA_val_xX_f = (DATA_val_xX_f.reshape(513,-1).T).reshape(-1,5,513)

In [7]:
print('DATA_train_s_f', DATA_train_s_f.shape)
print('DATA_train_M_f', DATA_train_M_f.shape)
print('DATA_val_x_f', DATA_val_x_f.shape)
print('DATA_test_x_f', DATA_test_x_f.shape)
print('DATA_val_xX_f', DATA_val_xX_f.shape)

DATA_train_s_f (23710, 5, 513)
DATA_train_M_f (23710, 5, 513)
DATA_val_x_f (23710, 5, 513)
DATA_test_x_f (8830, 5, 513)
DATA_val_xX_f (23710, 5, 513)


In [22]:
tr_categorical_labels = to_categorical(DATA_train_M_f, num_classes=2)
print(tr_categorical_labels.shape)

v_categorical_labels = to_categorical(DATA_val_M_f, num_classes=2)
print(v_categorical_labels.shape)

(23710, 5, 513, 2)
(23710, 5, 513, 2)


### RNN Model

In [8]:
Max_RNN = 5

In [10]:
# create the model

model = Sequential()

model.add( Bidirectional(GRU(Max_RNN, return_sequences=True), input_shape=(Max_RNN,513) ))
model.add(Dropout(0.1))
model.add(Dense(513, activation='sigmoid'))
          
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# model.fit(DATA_train_x_f.T, tr_categorical_labels, validation_data=(DATA_val_x_f.T,v_categorical_labels), shuffle=True, nb_epoch=100, batch_size=10)
# # Final evaluation of the model
# scores = model.evaluate(DATA_val_x_f.T, v_categorical_labels, verbose=0)



model.fit(DATA_train_x_f, DATA_train_M_f, validation_data=(DATA_val_x_f,DATA_val_M_f), shuffle=True, nb_epoch=10, batch_size=10)

# model.fit(DATA_train_x_f, tr_categorical_labels, validation_data=(DATA_val_x_f,v_categorical_labels), shuffle=True, nb_epoch=10, batch_size=10)
# Final evaluation of the model
scores = model.evaluate(DATA_val_x_f, DATA_val_M_f, verbose=0)



print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 5, 10)             15570     
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 10)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 5, 513)            5643      
Total params: 21,213
Trainable params: 21,213
Non-trainable params: 0
_________________________________________________________________
None


/opt/Anaconda3-5.0.1/envs/e533/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 23710 samples, validate on 23710 samples
Epoch 1/10
23710/23710 [==============================] - 55s 2ms/step - loss: 0.5898 - acc: 0.6839 - val_loss: 0.5429 - val_acc: 0.7291
Epoch 2/10
23710/23710 [==============================] - 52s 2ms/step - loss: 0.5468 - acc: 0.7229 - val_loss: 0.5234 - val_acc: 0.7439
Epoch 3/10
23710/23710 [==============================] - 52s 2ms/step - loss: 0.5346 - acc: 0.7325 - val_loss: 0.5102 - val_acc: 0.7529
Epoch 4/10
23710/23710 [==============================] - 53s 2ms/step - loss: 0.5267 - acc: 0.7385 - val_loss: 0.5061 - val_acc: 0.7552
Epoch 5/10
23710/23710 [==============================] - 53s 2ms/step - loss: 0.5212 - acc: 0.7429 - val_loss: 0.5035 - val_acc: 0.7576
Epoch 6/10
23710/23710 [==============================] - 52s 2ms/step - loss: 0.5175 - acc: 0.7459 - val_loss: 0.4956 - val_acc: 0.7625
Epoch 7/10
23710/23710 [==============================] - 52s 2ms/step - loss: 0.5145 - acc: 0.7477 - val_loss: 0.4921 - val_acc

In [ ]:
model = Sequential()

model.add(GRU(output_dim = 513, input_shape=(Max_RNN,513), return_sequences=True))
# model.add(GRU(output_dim = 513, input_length = 5, input_dim = 513, return_sequences=True))

model.add(Activation('relu'))
model.add(TimeDistributed(Dense(513, activation='sigmoid')))
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

model.fit(DATA_train_x_f, DATA_train_M_f, validation_data=(DATA_val_x_f,DATA_val_M_f), shuffle=True, epochs=10, batch_size=10)

# model.fit(DATA_train_x_f, tr_categorical_labels, validation_data=(DATA_val_x_f,v_categorical_labels), shuffle=True, nb_epoch=10, batch_size=10)
# Final evaluation of the model
scores = model.evaluate(DATA_val_x_f, DATA_val_M_f, verbose=0)



print("Accuracy: %.2f%%" % (scores[1]*100))

/opt/Anaconda3-5.0.1/envs/e533/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(input_shape=(5, 513), return_sequences=True, units=513)`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 5, 513)            1580553   
_________________________________________________________________
activation_1 (Activation)    (None, 5, 513)            0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 513)            263682    
Total params: 1,844,235
Trainable params: 1,844,235
Non-trainable params: 0
_________________________________________________________________
None
Train on 23710 samples, validate on 23710 samples
Epoch 1/10


In [8]:
def preprossed_dataX(file_name):
    
    sn, sr=librosa.load(file_name, sr=None)
    X=librosa.stft(sn, n_fft=1024, hop_length=512)
    return X

In [33]:
DATA_val_sX = None
DATA_val_xX = None

for file_s, file_x in zip( sorted(glob.glob(PATH_directory+PATH_val+CLEAN_format_val)), sorted(glob.glob(PATH_directory+PATH_val+MIX_format_val))):
    
    DATA_val_sX = np.array(preprossed_dataX(file_s)) if DATA_val_sX is None else np.concatenate( (DATA_val_sX,preprossed_dataX(file_s)),axis=1)
    DATA_val_xX = np.array(preprossed_dataX(file_x)) if DATA_val_xX is None else np.concatenate( (DATA_val_xX,preprossed_dataX(file_x)),axis=1)
    
    
    
    

In [10]:
print('DATA_val_sX',DATA_val_sX.shape)
print('DATA_val_xX',DATA_val_xX.shape)

DATA_val_sX = DATA_val_sX.reshape( (513,5,-1))
DATA_val_xX = DATA_val_xX.reshape( (513,5,-1))

print('DATA_val_sX',DATA_val_sX.shape)
print('DATA_val_xX',DATA_val_xX.shape)

DATA_val_xX (513, 118550)
DATA_val_xX (513, 5, 23710)


In [ ]:
np.save('DATA_val_sX.npy', DATA_val_sX)
np.save('DATA_val_xX.npy', DATA_val_xX)

In [13]:
DATA_val_sX_f = np.load('DATA_val_sX.npy')
DATA_val_xX_f = np.load('DATA_val_xX.npy')

In [25]:
labels = model.predict(DATA_test_x)
labels.shape

ValueError: Error when checking model : the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 400 arrays: [array([[0.31088, 0.00439, 0.02531, ..., 0.03555, 0.05921, 0.02048],
       [0.20403, 0.21771, 0.24856, ..., 0.07964, 0.19597, 0.11246],
       [0.00054, 0.27443, 0.5295 , ..., 0.14783, 0.20548, 0.119...

In [12]:
S_hat = (labels) * DATA_val_xX_f

In [13]:
# S_org = (DATA_val_s_f).flatten()
# S_pred = S_hat.flatten()
st = DATA_val_sX_f.reshape(513,-1)
st_hat = S_hat.reshape(513,-1)

print(st.shape)
print(st_hat.shape)

S_org = librosa.istft(st, hop_length=512)
S_pred = librosa.istft(st_hat, hop_length=512)

(513, 118550)
(513, 118550)


In [20]:
print(S_pred.shape, S_org.shape)

(60697088,) (60697088,)


In [14]:
acc = np.sum(S_org*S_org)/ np.sum((S_org-S_pred)*(S_org-S_pred))
print(acc)

10*np.log10(acc)

4.1112037


6.139689683914185

In [1]:
# Noisy -> Input Data
sn, sr=librosa.load('train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)
X_mag = np.abs(X)

# Clean -> Label
s, sr=librosa.load('train_clean_male.wav', sr=None)
S=librosa.stft(s, n_fft=1024, hop_length=512)
S_mag = np.abs(S)

IndentationError: expected an indented block (<ipython-input-1-09b5a3680f5d>, line 9)

### Function for generating next batch

In [56]:
def next_batch(X,Y, batch_size):
    num_samples, _ = X.shape
    
    selected_indics = np.random.randint(num_samples-batch_size)
#     print(selected_indics)
    return X[selected_indics:selected_indics+batch_size], Y[selected_indics:selected_indics+batch_size]

### Global Parameters

In [57]:
NUM_ITERATION = 1000
BATCH_SIZE = X_mag.shape[0]

### Xavier Initialization of Weights
These are the weight initialization function used in defining model.

In [58]:
def weight_variable (shape):
    initial = tf.truncated_normal(shape, stddev = np.sqrt(2.0/sum(shape)) )
    return tf.Variable(initial)

def bias_variable (shape):
    initial = tf.truncated_normal(shape, stddev = np.sqrt(1.0/sum(shape)) )
    return tf.Variable(initial)

### Create the fully connected model 

In [59]:
x = tf.placeholder(tf.float32, [None, 513]) 

W_1 = weight_variable([513, 1024])
b_1 = bias_variable([1024])

W_2 = weight_variable([1024, 1024])
b_2 = bias_variable([1024])

W_3 = weight_variable([1024, 1024])
b_3 = bias_variable([1024])

W_4 = weight_variable([1024, 1024])
b_4 = bias_variable([1024])

W_5 = weight_variable([1024, 513])
b_5 = bias_variable([513])

y_ = tf.placeholder(tf.float32, [None, 513]) # original


# Layer connections and Activation functions
y_1 = tf.nn.relu(tf.matmul(x, W_1) + b_1)
y_2 = tf.nn.relu(tf.matmul(y_1, W_2) + b_2)
y_3 = tf.nn.relu(tf.matmul(y_2, W_3) + b_3)
y_4 = tf.nn.relu(tf.matmul(y_3, W_4) + b_4)
y =  tf.nn.relu(tf.matmul(y_2, W_5) + b_5) # predicted


# Define loss and optimizer
mse = tf.reduce_sum( tf.losses.mean_squared_error(labels=y_, predictions=y) )
train_step = tf.train.AdamOptimizer().minimize(mse)

### Run the model
We try different batch sizes, but for whole input dimension batch size gives better result and it removes the need of batch normalization. Since the input dimension is not very large, we can do this. For larger input dimensions, we have to adopt mini batch techniques. 

In [60]:
# Configuration to control GPU use
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.33
sess = tf.InteractiveSession(config=config)

tf.global_variables_initializer().run()


# Train Model
for _ in range(NUM_ITERATION):
    for _ in range(((X_mag.T).shape[0]//BATCH_SIZE)):
        batch_xs, batch_ys = next_batch(X_mag.T,S_mag.T, BATCH_SIZE)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

### Read the Test files

In [61]:
# Load Test data-1
sn, sr=librosa.load('test_x_01.wav', sr=None)
X_test_01=librosa.stft(sn, n_fft=1024, hop_length=512)
X_mag_test_01 = np.abs(X_test_01)

# Load Test data-2
sn, sr=librosa.load('test_x_02.wav', sr=None)
X_test_02=librosa.stft(sn, n_fft=1024, hop_length=512)
X_mag_test_02 = np.abs(X_test_02)

print(X_mag_test_01.shape)

(513, 142)


### Save the Test Output Files

In [62]:
# Test model-1
S_hat_mag_test_01=sess.run(y, feed_dict={x: X_mag_test_01.T})
S_hat_test_01=(X_test_01/X_mag_test_01)*S_hat_mag_test_01.T
S_hat_01=librosa.istft(S_hat_test_01, hop_length=512)
librosa.output.write_wav('test_s_01_recons.wav', S_hat_01, sr)

# Test model-2
S_hat_mag_test_02=sess.run(y, feed_dict={x: X_mag_test_02.T})
S_hat_test_02=(X_test_02/X_mag_test_02)*S_hat_mag_test_02.T
S_hat_02=librosa.istft(S_hat_test_02, hop_length=512)
librosa.output.write_wav('test_s_02_recons.wav', S_hat_02, sr)